In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("siddharthm1698/coursera-course-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/coursera-course-dataset


In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

class SkillGapAnalyzer:
    def __init__(self, coursera_dataset_path, debug=False):
        """
        Initialize với dataset Coursera từ Kaggle
        Dataset cần có các cột: 'course_name', 'skills', 'course_url', etc.
        """
        self.courses_df = pd.read_csv(coursera_dataset_path)
        self.debug = debug
        
        if debug:
            print("📊 Dataset Info:")
            print(f"Columns: {self.courses_df.columns.tolist()}")
            print(f"Shape: {self.courses_df.shape}")
            print("\nFirst row sample:")
            print(self.courses_df.iloc[0])
            print("\n" + "="*80 + "\n")
        
        self.vectorizer = TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=1000,
            stop_words='english',
            min_df=1  # Cho phép từ xuất hiện ít nhất 1 lần
        )
        self._preprocess_courses()
        
    def _preprocess_courses(self):
        """Tiền xử lý dataset khóa học"""
        # Tìm cột skills trong dataset - mở rộng tìm kiếm
        skill_column = None
        possible_names = ['skills', 'Skills', 'skill', 'Skill', 'course_skills', 
                          'Course Skills', 'what_you_will_learn', 'description',
                          'course_description', 'learn']
        
        for col in possible_names:
            if col in self.courses_df.columns:
                skill_column = col
                break
        
        # Nếu vẫn không tìm thấy, tìm cột có chứa từ 'skill' hoặc 'learn'
        if not skill_column:
            for col in self.courses_df.columns:
                if 'skill' in col.lower() or 'learn' in col.lower() or 'describe' in col.lower():
                    skill_column = col
                    break
        
        # Xử lý cột skills
        if skill_column:
            # Chuyển tất cả về string và xử lý NaN
            self.courses_df['skills_clean'] = (
                self.courses_df[skill_column]
                .fillna('')
                .astype(str)
                .str.lower()
                .str.strip()
            )
            if self.debug:
                print(f"✅ Using column: '{skill_column}'")
                print(f"Sample values:\n{self.courses_df['skills_clean'].head(3)}\n")
        else:
            # Nếu không có cột skills, kết hợp nhiều cột
            print("⚠️  Warning: No clear 'skills' column found.")
            print(f"Available columns: {self.courses_df.columns.tolist()}")
            
            # Thử kết hợp tên khóa học + cột đầu tiên
            text_cols = []
            for col in self.courses_df.columns[:5]:  # Lấy 5 cột đầu
                if self.courses_df[col].dtype == 'object':
                    text_cols.append(col)
            
            if text_cols:
                self.courses_df['skills_clean'] = (
                    self.courses_df[text_cols].fillna('')
                    .astype(str)
                    .agg(' '.join, axis=1)
                    .str.lower()
                    .str.strip()
                )
                print(f"Using combined columns: {text_cols}")
        
        # Loại bỏ các dòng có skills rỗng
        original_count = len(self.courses_df)
        self.courses_df = self.courses_df[self.courses_df['skills_clean'] != ''].reset_index(drop=True)
        
        print(f"✅ Loaded {len(self.courses_df)} courses (removed {original_count - len(self.courses_df)} empty rows)")
        
        # Fit vectorizer trên toàn bộ skills
        if len(self.courses_df) > 0:
            self.course_vectors = self.vectorizer.fit_transform(self.courses_df['skills_clean'])
        else:
            raise ValueError("❌ No valid courses found in dataset!")
    
    def normalize_skills(self, skills_list):
        """
        Chuẩn hóa skills để xử lý các từ khóa khác nhau
        """
        normalized = []
        skill_mapping = {
            # Programming languages
            'python': ['python', 'python3', 'py'],
            'javascript': ['javascript', 'js', 'node.js', 'nodejs'],
            'java': ['java', 'java programming'],
            'sql': ['sql', 'mysql', 'postgresql', 'database'],
            
            # Data Science
            'machine learning': ['machine learning', 'ml', 'deep learning', 'ai'],
            'data analysis': ['data analysis', 'data analytics', 'analytics'],
            'data visualization': ['data visualization', 'tableau', 'power bi', 'visualization'],
            
            # Web Development
            'html': ['html', 'html5'],
            'css': ['css', 'css3', 'styling'],
            'react': ['react', 'reactjs', 'react.js'],
            
            # Cloud & DevOps
            'aws': ['aws', 'amazon web services', 'cloud'],
            'docker': ['docker', 'containerization', 'containers'],
            'kubernetes': ['kubernetes', 'k8s', 'orchestration'],
            
            # Soft Skills
            'communication': ['communication', 'verbal communication', 'presentation'],
            'leadership': ['leadership', 'team management', 'management'],
            'project management': ['project management', 'agile', 'scrum'],
        }
        
        for skill in skills_list:
            skill_lower = skill.lower().strip()
            matched = False
            
            for standard_skill, variants in skill_mapping.items():
                if any(variant in skill_lower for variant in variants):
                    if standard_skill not in normalized:
                        normalized.append(standard_skill)
                    matched = True
                    break
            
            if not matched:
                normalized.append(skill_lower)
        
        return normalized
    
    def find_skill_gaps(self, user_skills, required_skills):
        """
        Tìm kỹ năng còn thiếu
        """
        # Chuẩn hóa skills
        user_skills_norm = set(self.normalize_skills(user_skills))
        required_skills_norm = set(self.normalize_skills(required_skills))
        
        # Tìm skills còn thiếu
        missing_skills = required_skills_norm - user_skills_norm
        
        return list(missing_skills)
    
    def recommend_courses(self, missing_skills, top_n=5):
        """
        Gợi ý khóa học dựa trên skills còn thiếu
        """
        if not missing_skills:
            return pd.DataFrame()
        
        # Tạo query từ missing skills
        query = ' '.join(missing_skills)
        
        if self.debug:
            print(f"\n🔍 Query: {query}")
        
        query_vector = self.vectorizer.transform([query])
        
        # Tính cosine similarity
        similarities = cosine_similarity(query_vector, self.course_vectors).flatten()
        
        if self.debug:
            print(f"Similarity range: {similarities.min():.4f} to {similarities.max():.4f}")
            print(f"Non-zero similarities: {(similarities > 0).sum()}")
        
        # Lấy top N khóa học
        top_indices = similarities.argsort()[-top_n:][::-1]
        
        # Tạo DataFrame kết quả
        recommendations = self.courses_df.iloc[top_indices].copy()
        recommendations['relevance_score'] = similarities[top_indices]
        recommendations['missing_skills_matched'] = recommendations['skills_clean'].apply(
            lambda x: [skill for skill in missing_skills if skill in str(x).lower()]
        )
        
        return recommendations
    
    def generate_report(self, user_skills, required_skills, top_n=5):
        """
        Tạo báo cáo hoàn chỉnh
        """
        print("=" * 80)
        print("SKILL GAP ANALYSIS REPORT")
        print("=" * 80)
        
        # 1. Hiển thị skills hiện tại
        print("\n📊 YOUR CURRENT SKILLS:")
        for skill in user_skills:
            print(f"  ✓ {skill}")
        
        # 2. Hiển thị skills yêu cầu
        print("\n🎯 REQUIRED SKILLS:")
        for skill in required_skills:
            print(f"  • {skill}")
        
        # 3. Tìm skill gaps
        missing_skills = self.find_skill_gaps(user_skills, required_skills)
        
        print("\n⚠️  MISSING SKILLS:")
        if missing_skills:
            for skill in missing_skills:
                print(f"  ✗ {skill}")
        else:
            print("  🎉 Congratulations! You have all required skills!")
            return None
        
        # 4. Gợi ý khóa học
        recommendations = self.recommend_courses(missing_skills, top_n)
        
        print(f"\n📚 TOP {top_n} RECOMMENDED COURSES:")
        print("-" * 80)
        
        # Tìm cột tên khóa học
        name_columns = ['course_name', 'Course Name', 'name', 'Name', 'title', 'Title', 'course_title']
        course_name_col = None
        
        for col in name_columns:
            if col in recommendations.columns:
                course_name_col = col
                break
        
        # Nếu không tìm thấy, dùng cột đầu tiên
        if not course_name_col:
            course_name_col = recommendations.columns[0]
        
        # Tìm cột URL
        url_columns = ['course_url', 'Course URL', 'url', 'URL', 'link', 'Link']
        url_col = None
        for col in url_columns:
            if col in recommendations.columns:
                url_col = col
                break
        
        for idx, (_, row) in enumerate(recommendations.iterrows(), 1):
            course_name = row.get(course_name_col, 'Unknown Course')
            print(f"\n{idx}. {course_name}")
            print(f"   Relevance Score: {row['relevance_score']:.3f}")
            
            if row['missing_skills_matched']:
                print(f"   Skills Covered: {', '.join(row['missing_skills_matched'])}")
            else:
                print(f"   Skills Covered: (check course details)")
            
            if url_col and pd.notna(row.get(url_col)):
                print(f"   URL: {row[url_col]}")
        
        print("\n" + "=" * 80)
        
        return recommendations


# ===== EXAMPLE USAGE =====
if __name__ == "__main__":
    # 1. Load dataset từ Kaggle với debug mode
    analyzer = SkillGapAnalyzer('/kaggle/input/coursera-course-dataset/coursea_data.csv', debug=True)
    
    # 2. Define user skills and required skills
    user_skills = [
        'Python',
        'HTML',
        'CSS',
        'SQL'
    ]
    
    required_skills = [
        'Python',
        'Machine Learning',
        'Deep Learning',
        'SQL',
        'AWS',
        'Docker',
        'React'
    ]
    
    # 3. Generate report
    recommendations = analyzer.generate_report(user_skills, required_skills, top_n=5)
    
    # 4. Save recommendations to CSV
    if recommendations is not None:
        recommendations.to_csv('recommended_courses.csv', index=False)
        print("\n✅ Recommendations saved to 'recommended_courses.csv'")

📊 Dataset Info:
Columns: ['Unnamed: 0', 'course_title', 'course_organization', 'course_Certificate_type', 'course_rating', 'course_difficulty', 'course_students_enrolled']
Shape: (891, 7)

First row sample:
Unnamed: 0                                                                134
course_title                (ISC)² Systems Security Certified Practitioner...
course_organization                                                    (ISC)²
course_Certificate_type                                        SPECIALIZATION
course_rating                                                             4.7
course_difficulty                                                    Beginner
course_students_enrolled                                                 5.3k
Name: 0, dtype: object


⚠️  Warning: No clear 'skills' column found.
Available columns: ['Unnamed: 0', 'course_title', 'course_organization', 'course_Certificate_type', 'course_rating', 'course_difficulty', 'course_students_enrolled']
Using combi